Read Collection Pickle

In [74]:
import numpy as np
import pandas as pd

# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = '../data/subtask_4b/subtask4b_collection_data.pkl' #MODIFY PATH
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [75]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


Process Query File

In [76]:
PATH_QUERY_DATA = '../data/subtask_4b/subtask4b_query_tweets_train.tsv'
df_query = pd.read_csv(PATH_QUERY_DATA, sep = '\t')
df_query.head()

,post_id,tweet_text,cord_uid
0,0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,1,this study isn't receiving sufficient attentio...,4kfl29ul
2,2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [77]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12853 entries, 0 to 12852
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     12853 non-null  int64 
 1   tweet_text  12853 non-null  object
 2   cord_uid    12853 non-null  object
dtypes: int64(1), object(2)
memory usage: 301.4+ KB


In [78]:
FILTERED_SAMPLES_WITH_GOLD_PATH = "../data/pairwise-model-data/gold_pred.tsv"
df_filtered_samples = pd.read_csv(FILTERED_SAMPLES_WITH_GOLD_PATH, sep = '\t')
df_filtered_samples.head()

,post_id,cord_uid,preds
0,0,htlvpvz5,"['htlvpvz5', 'vccct6hq', 'fkwgq5mr', 'znpc5jq2..."
1,2,jtwb17u8,"['jtwb17u8', 'iobpcfs5', 'veeavho5', '2tu707ng..."
2,3,0w9k8iy1,"['0w9k8iy1', 'l4y7v729', 'q5wiqpcb', 'gy0kfhy6..."
3,4,tiqksd69,"['tiqksd69', 'aqbhxv1f', 'mqu6d8sp', 'lr5lumdr..."
4,5,ut1k8xe7,"['ut1k8xe7', '0chuwvg6', 'rvygtpvb', 'eksi9yia..."


In [79]:
import random
from numpy import result_type
from tqdm import tqdm

tqdm.pandas()

def flatten(entry):    
    post_id = entry["post_id"]
    gold_uid = entry["cord_uid"]
    predictions = eval(entry["preds"])
    # predictions = predictions_string.split(",").remove(gold_uid)
    tweet = df_query[df_query["post_id"] == post_id]["tweet_text"]
    predictions.remove(gold_uid)
    positive_doc = df_collection[df_collection['cord_uid'] == gold_uid]
    negative_docs = df_collection[df_collection['cord_uid'].isin(predictions)]

    queries = [tweet.values[0]]*8
    right_abstracts = []
    left_abstracts = []
    labels = [0, 1]*4
    
    for pred_uid in predictions:
        paper_data = "{authors} [SEP] {title} [SEP] {journal} [SEP] {abstract}"
        negative_doc = negative_docs[negative_docs["cord_uid"] == pred_uid]

        neg_paper_data = paper_data.format(authors=negative_doc["authors"].values[0],
                                           title=negative_doc["title"].values[0],
                                           journal=negative_doc["journal"].values[0],
                                           abstract=negative_doc["abstract"].values[0],
                                           )
        
        pos_paper_data = paper_data.format(authors=positive_doc["authors"].values[0],
                                           title=positive_doc["title"].values[0],
                                           journal=positive_doc["journal"].values[0],
                                           abstract=positive_doc["abstract"].values[0],
                                           )
        
        right_abstracts.append(pos_paper_data)
        right_abstracts.append(neg_paper_data)

        left_abstracts.append(neg_paper_data)
        left_abstracts.append(pos_paper_data)
    return {"query": queries, "paper1": right_abstracts, "paper2": left_abstracts, "label":labels}

pairwise_train_data = df_filtered_samples.progress_apply(lambda x: flatten(x), axis = 1, result_type="expand").explode(["query", "paper1", "paper2", "label"])
pairwise_train_data.head()

  0%|          | 0/1544 [00:00<?, ?it/s]

100%|██████████| 1544/1544 [00:03<00:00, 387.50it/s]


,query,paper1,paper2,label
0,Oral care in rehabilitation medicine: oral vul...,"Shiraishi, A.; Wakabayashi, Hidetaka; Yoshimur...","Rogers-Brown, Jessica S.; Wanga, Valentine; Ok...",0
0,Oral care in rehabilitation medicine: oral vul...,"Rogers-Brown, Jessica S.; Wanga, Valentine; Ok...","Shiraishi, A.; Wakabayashi, Hidetaka; Yoshimur...",1
0,Oral care in rehabilitation medicine: oral vul...,"Shiraishi, A.; Wakabayashi, Hidetaka; Yoshimur...","Wiertz, Carolina M.H.; Vints, Wouter A.J.; Maa...",0
0,Oral care in rehabilitation medicine: oral vul...,"Wiertz, Carolina M.H.; Vints, Wouter A.J.; Maa...","Shiraishi, A.; Wakabayashi, Hidetaka; Yoshimur...",1
0,Oral care in rehabilitation medicine: oral vul...,"Shiraishi, A.; Wakabayashi, Hidetaka; Yoshimur...","Huang, Ni; Pérez, Paola; Kato, Takafumi; Mikam...",0


In [80]:
pairwise_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12352 entries, 0 to 1543
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   query   12352 non-null  object
 1   paper1  12352 non-null  object
 2   paper2  12352 non-null  object
 3   label   12352 non-null  object
dtypes: object(4)
memory usage: 482.5+ KB


In [81]:
pairwise_train_data.to_csv('../data/pairwise-model-data/pairedwise-tweets.tsv',sep='\t')